# A2A Multi-Agent Customer Support System Demo

This notebook demonstrates a multi-agent system using Google ADK and A2A protocol.

## System Architecture
- **Router Agent**: Orchestrates queries and routes to specialist agents
- **Customer Data Agent**: Handles database operations via MCP
- **Support Agent**: Manages customer support queries and tickets

## Prerequisites
1. MCP Server running on `http://127.0.0.1:8000`
2. Agent servers running on ports 10020, 10021, 10022
3. Run `python agent_server.py` in a separate terminal

In [15]:
# Import required libraries
import asyncio
import httpx
from termcolor import colored
from a2a.types import AgentCard, TransportProtocol
from a2a.client import ClientConfig, ClientFactory, create_text_message_object
from a2a.utils.constants import AGENT_CARD_WELL_KNOWN_PATH
from IPython.display import display, Markdown, HTML
import time

In [16]:
import nest_asyncio
nest_asyncio.apply()
print("✅ Async support enabled")

✅ Async support enabled


In [17]:
# Configuration
ROUTER_URL = "http://127.0.0.1:10022"

async def query_router(scenario, query_text, show_details=False):
    """
    Send a query to the router agent and display the response.
    
    Args:
        scenario: Description of the test scenario
        query_text: The query to send
        show_details: Whether to show intermediate responses
    """
    print("="*80)
    display(Markdown(f"## 🎬 {scenario}"))
    display(Markdown(f"**Query:** `{query_text}`"))
    print()
    
    start_time = time.time()
    
    async with httpx.AsyncClient(timeout=120.0) as client:
        try:
            # Fetch Router Card
            resp = await client.get(f"{ROUTER_URL}{AGENT_CARD_WELL_KNOWN_PATH}")
            if resp.status_code != 200:
                display(Markdown("❌ **Error:** Router unavailable. Make sure `agent_server.py` is running."))
                return
            
            card = AgentCard(**resp.json())
            
            # Setup Client
            factory = ClientFactory(ClientConfig(
                httpx_client=client, 
                supported_transports=[TransportProtocol.jsonrpc]
            ))
            a2a = factory.create(card)
            
            print("⚡ Processing...")
            msg = create_text_message_object(content=query_text)
            
            # Collect responses
            responses = []
            async for response in a2a.send_message(msg):
                if isinstance(response, tuple) and response:
                    try:
                        answer = response[0].artifacts[0].parts[0].root.text
                        responses.append(answer)
                        
                        if show_details and len(responses) > 1:
                            display(Markdown(f"*Intermediate response {len(responses)-1}:*"))
                            print(answer[:200] + "..." if len(answer) > 200 else answer)
                            print()
                    except Exception as e:
                        if show_details:
                            print(f"Parse error: {e}")
            
            elapsed = time.time() - start_time
            
            # Display final response
            if responses:
                if len(responses) > 1:
                    display(Markdown(f"*ℹ️ Received {len(responses)} responses from agent chain*"))
                
                display(Markdown("### 🤖 Response:"))
                display(Markdown(responses[-1]))
                print(f"\n⏱️ Time taken: {elapsed:.2f}s")
            else:
                display(Markdown("⚠️ **Warning:** No text response received"))
                
        except Exception as e:
            display(Markdown(f"❌ **Error:** {str(e)}"))
            import traceback
            if show_details:
                traceback.print_exc()
    
    print("\n")

---
## Test Scenario 1: Simple Query

**Goal:** Test single agent data retrieval  
**Expected Flow:** Router → Data Agent → MCP Server → Response

In [18]:
await query_router(
    scenario="Simple Query - Data Retrieval",
    query_text="Get customer information for ID 5",
    show_details=False
)

## 🎬 Simple Query - Data Retrieval

**Query:** `Get customer information for ID 5`


⚡ Processing...


*ℹ️ Received 2 responses from agent chain*

### 🤖 Response:

Customer ID 5 is Charlie Brown, active, email charlie.brown@email.com, phone +1-555-0105. Is there anything else?


⏱️ Time taken: 2.34s




---
## Test Scenario 2: Coordinated Query

**Goal:** Test multi-agent coordination  
**Expected Flow:** Router → Data Agent (fetch customer) → Support Agent (provide help) → Combined Response

In [19]:
await query_router(
    scenario="Coordinated Query - Multi-Agent",
    query_text="I'm customer 12345 and need help upgrading my account",
    show_details=True
)

## 🎬 Coordinated Query - Multi-Agent

**Query:** `I'm customer 12345 and need help upgrading my account`


⚡ Processing...
Parse error: 'NoneType' object is not subscriptable
Parse error: 'NoneType' object is not subscriptable
Parse error: 'NoneType' object is not subscriptable
Parse error: 'NoneType' object is not subscriptable


*Intermediate response 1:*

I am sorry, but I cannot find a customer with the ID 12345. Please ensure that you have provided the correct customer ID. If you are unsure, I can help you find your customer ID if you can provide me ...



*ℹ️ Received 2 responses from agent chain*

### 🤖 Response:

I am sorry, but I cannot find a customer with the ID 12345. Please ensure that you have provided the correct customer ID. If you are unsure, I can help you find your customer ID if you can provide me with your name.


⏱️ Time taken: 2.23s




---
## Test Scenario 3: Complex Query

**Goal:** Test complex data correlation requiring multiple MCP calls  
**Expected Flow:** Router coordinates Data Agent to filter customers by status AND ticket status

In [20]:
await query_router(
    scenario="Complex Query - Data Correlation",
    query_text="Show me all active customers who have open tickets",
    show_details=False
)

## 🎬 Complex Query - Data Correlation

**Query:** `Show me all active customers who have open tickets`


⚡ Processing...


*ℹ️ Received 2 responses from agent chain*

### 🤖 Response:

The following customers have open tickets: John Doe, Edward Norton, Charlie Brown, George Miller, Michael Scott, Jane Smith, Diana Prince, Isaac Newton, Julia Roberts, Alice Williams, and Hannah Lee.

All of these customers are active.


⏱️ Time taken: 3.88s




---
## Test Scenario 4: Escalation

**Goal:** Test urgency detection and appropriate routing  
**Expected Flow:** Router detects urgency → Support Agent handles with priority escalation

In [25]:
await query_router(
    scenario="Escalation - Urgent Issue",
    query_text="I've been charged twice, please refund immediately!",
    show_details=False
)

## 🎬 Escalation - Urgent Issue

**Query:** `I've been charged twice, please refund immediately!`


⚡ Processing...


*ℹ️ Received 2 responses from agent chain*

### 🤖 Response:

I understand you're upset about being charged twice, but I am unable to process refunds directly. The available tools do not support this function.

However, I can assist you by:
* Creating a support ticket to document this issue.
* Providing details about your customer account, if you can provide your customer ID.

Could you please provide your customer ID so I can create a ticket for you?


⏱️ Time taken: 1.99s




---
## Test Scenario 5: Multi-Intent Query

**Goal:** Test parallel task execution  
**Expected Flow:** Router decomposes into: (1) Update email via Data Agent, (2) Get history via Data Agent  

**Note:** This requires a customer ID. Let's assume customer ID 1.

In [26]:
await query_router(
    scenario="Multi-Intent Query - Parallel Tasks",
    query_text="I am customer 1. Update my email to newemail@example.com and show my ticket history",
    show_details=True
)

## 🎬 Multi-Intent Query - Parallel Tasks

**Query:** `I am customer 1. Update my email to newemail@example.com and show my ticket history`


⚡ Processing...
Parse error: 'NoneType' object is not subscriptable
Parse error: 'NoneType' object is not subscriptable
Parse error: 'NoneType' object is not subscriptable
Parse error: 'NoneType' object is not subscriptable


*Intermediate response 1:*

I have updated your email to newemail@example.com. Here is your ticket history:

* Ticket ID: 1, Issue: Cannot login to account, Status: open, Priority: high, Created: 2025-12-07 20:52:51
* Ticket ID:...



*ℹ️ Received 2 responses from agent chain*

### 🤖 Response:

I have updated your email to newemail@example.com. Here is your ticket history:

* Ticket ID: 1, Issue: Cannot login to account, Status: open, Priority: high, Created: 2025-12-07 20:52:51
* Ticket ID: 6, Issue: Password reset not working, Status: in_progress, Priority: medium, Created: 2025-12-07 20:52:51


⏱️ Time taken: 3.41s




---
## Batch Testing

Run all test scenarios in sequence:

In [30]:
test_scenarios = [
    ("Simple Query", "Get customer information for ID 5"),
    ("Coordinated Query", "I'm customer 12345 and need help upgrading my account"),
    ("Complex Query", "Show me all active customers who have open tickets"),
    ("Escalation", "I've been charged twice, please refund immediately!"),
    ("Multi-Intent", "I am customer 1. Update my email to newemail@example.com and show my ticket history"),
]

for scenario, query in test_scenarios:
    await query_router(scenario, query, show_details=False)
    await asyncio.sleep(1)  # Brief pause between queries

## 🎬 Simple Query

**Query:** `Get customer information for ID 5`


⚡ Processing...


*ℹ️ Received 2 responses from agent chain*

### 🤖 Response:

...


⚡ Processing...


*ℹ️ Received 2 responses from agent chain*

### 🤖 Response:

Your email has been updated to newemail@example.com.

Here is your ticket history:

- Ticket ID: 29, Issue: Double charge, refund immediately., Status: open, Priority: high, Created: 2025-12-07 21:33:09
- Ticket ID: 1, Issue: Cannot login to account, Status: open, Priority: high, Created: 2025-12-07 20:52:51
- Ticket ID: 6, Issue: Password reset not working, Status: in_progress, Priority: medium, Created: 2025-12-07 20:52:51



⏱️ Time taken: 2.62s


